<a href="https://colab.research.google.com/github/sandroklippel/misc_noteboooks/blob/main/Historico_Mapbiomas_de_um_Alerta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Histórico de Uso e Cobertura do Solo de um alerta Mapbiomas

Instruções:

1.   O polígono do alerta será obtido diretamente do servidor Mapbiomas via API portanto será necessário usar as credencias de acesso. Se você não tem uma conta, cadastre-se no site https://plataforma.alerta.mapbiomas.org/sign-up.
2. Preencha o formulário abaixo com essas credenciais, bem como o número do alerta. 
3. Atribua o nome do projeto registrado no Google Earth Engine na variável `PROJECT`. Você encontra o nome do projeto em code.earthengine.google.com como no exemplo abaixo: 
![image.png](image.png)
4.   Selecione no menu "Ambiente de execução" -> "Executar tudo" ou pressione Ctrl+F9.
5. O gráfico será gerado na última célula. Há opção para salvá-lo em .PNG.



In [58]:
#@title Dados de entrada { display-mode: "form" }
email = "someone@somewhere" #@param {type:"string"}
senha = "password1234" #@param {type:"string"}
alerta =  12345#@param {type:"integer"}


In [ ]:
PROJECT=''

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [59]:
import ee
ee.Authenticate()
ee.Initialize(project=PROJECT)

In [60]:
import datetime
import requests
import pandas as pd
import numpy as np
import plotly.express as px

from shapely import wkt
from shapely.geometry import mapping


def busca_alerta(email, senha, alerta):
    """
    Busca o poligono do alerta no servidor Mapbiomas via API
    """

    ENDPOINT = 'https://plataforma.alerta.mapbiomas.org/api/graphql'
    TIMEOUT = 120

    MUTATION = \
    """mutation($email: String!, $password: String!)
       { 
         signIn(email: $email, password: $password)
        {
         token
        }
       }
    """

    QUERY = \
    """
    query
    (
      $alertCode: Int!
      $carId: Int
    )
    {
      alertReport(alertCode: $alertCode, carId: $carId) {
        alertCode
        alertGeomWkt
        areaHa
        }
    }
    """

    credentials = {'email': email, 'password': senha}

    request = requests.post(ENDPOINT, json={'query': MUTATION, 'variables': credentials}, timeout=TIMEOUT)

    if request.status_code == 200:
        result = request.json()
        if 'errors' in result:
            msg = '\n'.join([error['message'] for error in result['errors']])
            raise Exception("Falha na obtenção do alerta. Mensagem retornada: {}.".format(msg))
        else:
            token = result["data"]["signIn"]["token"]
    else:
        raise Exception("Falha na obtenção do alerta. Código retornado: {}.".format(request.status_code))

    headers = {'Authorization': 'Bearer ' + token}

    request = requests.post(ENDPOINT, json={'query': QUERY, 'variables':  {'alertCode': alerta}}, headers=headers, timeout=TIMEOUT)

    if request.status_code == 200:
        row = request.json()
        if 'errors' in row:
            msg = '\n'.join([error['message'] for error in row['errors']])
            raise Exception("Falha na obtenção do alerta. Mensagem retornada: {}.".format(msg))
    else:
        raise Exception("Falha na obtenção do alerta. Código retornado: {}.".format(request.status_code))

    wkt_string = row['data']['alertReport']['alertGeomWkt']
    shapely_geom = wkt.loads(wkt_string)
    return shapely_geom

def map_function(s):
  def mbyr(y):
      k = ee.String('classification_').cat(ee.String(y))
      d = ee.Dictionary(s.get(k))
      return d.set('year', y)
  return mbyr

classes = {1: 'Floresta',
           3: 'Formação Florestal',
           4: 'Formação Savânica',
           5: 'Mangue',
           6: 'Floresta Alagável',
           9: 'Silvicultura',
           10: 'Vegetação Herbácea e Arbustiva',
           11: 'Campo Alagado e Área Pantanosa',
           12: 'Formação Campestre',
           14: 'Agropecuária',
           15: 'Pastagem',
           18: 'Agricultura',
           19: 'Lavoura Temporária',
           20: 'Cana',
           21: 'Mosaico de Usos',
           22: 'Área não Vegetada',
           23: 'Praia, Duna e Areal',
           24: 'Área Urbanizada',
           25: 'Outras Áreas não Vegetadas',
           26: "Corpo D'água",
           27: 'Não observado',
           29: 'Afloramento Rochoso',
           30: 'Mineração',
           31: 'Aquicultura',
           32: 'Apicum',
           33: 'Rio, Lago e Oceano',
           35: 'Dendê',
           36: 'Lavoura Perene',
           39: 'Soja',
           40: 'Arroz',
           41: 'Outras Lavouras Temporárias',
           46: 'Café',
           47: 'Citrus',
           48: 'Outras Lavouras Perenes',
           49: 'Restinga Arbórea',
           50: 'Restinga Herbácea',
           62: 'Algodão (beta)',
           0: 'Não observado'}

cores =  {1: '#32a65e',
          3: '#1f8d49',
          4: '#7dc975',
          5: '#04381d',
          6: '#026975',
          9: '#7a5900',
          10: '#ad975a',
          11: '#519799',
          12: '#d6bc74',
          14: '#FFFFB2',
          15: '#edde8e',
          18: '#E974ED',
          19: '#C27BA0',
          20: '#db7093',
          21: '#ffefc3',
          22: '#d4271e',
          23: '#ffa07a',
          24: '#d4271e',
          25: '#db4d4f',
          26: '#0000FF',
          27: '#ffffff',
          29: '#ffaa5f',
          30: '#9c0027',
          31: '#091077',
          32: '#fc8114',
          33: '#2532e4',
          35: '#9065d0',
          36: '#d082de',
          39: '#f5b3c8',
          40: '#c71585',
          41: '#f54ca9',
          46: '#d68fe2',
          47: '#9932cc',
          48: '#e6ccff',
          49: '#02d659',
          50: '#ad5100',
          62: '#ff69b4',
          0:  '#ffffff'}

color_map = {classes[k]: v for k, v in cores.items()}

shapely_geom = busca_alerta(email, senha, alerta)
geojson = mapping(shapely_geom)

hoje = datetime.datetime.today().strftime('%d/%m/%Y')
geom = ee.Geometry(geojson)
area_ha = geom.area().getInfo() / 10000.0

mapbiomas = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1')

stats = (mapbiomas.reduceRegion(
         reducer = ee.Reducer.frequencyHistogram(),
        geometry = geom,
        scale = 30,
        crs = 'EPSG:4326'
        ))

years = ee.List([str(i) for i in range(1985,2024)])

map_by_year = map_function(stats)

lst = years.map(map_by_year)

df = pd.DataFrame(lst.getInfo())
df.set_index('year', inplace=True)
df_perc = df.div(df.iloc[0,:].sum(), axis=1).mul(100).round(2)
df_perc.columns = [classes[int(c)] for c in df_perc.columns.tolist()]
df_perc = df_perc.reindex(columns=sorted(df_perc.columns))

In [ ]:
fig = px.bar(df_perc, barmode='stack', 
             color_discrete_map=color_map, 
             labels={"value": "% da área do polígono", "variable": "", "year": ""},
             title=f"Alerta {alerta} ({area_ha:.2f} ha) - Histórico de Uso e Cobertura do Solo")

fig.update_layout(template='simple_white')
fig.update_xaxes(tickangle=-90)
fig.add_annotation(
    text = f"Fonte dos dados: Projeto MapBiomas - Coleção 9 da Série Anual de Mapas de Uso e Cobertura da Terra do Brasil, acesso em {hoje}.",
    showarrow=False,
    x = 0,
    y = -0.2,
    xref='paper',
    yref='paper',
    xanchor='left',
    yanchor='bottom',
    xshift=-1,
    yshift=-5,
    font=dict(size=10, color="grey"),
    align="left")
fig.show()

### Polígono para simples conferência:

In [ ]:
shapely_geom